In [1]:
!pip install gensim

  Using cached gensim-4.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.1 MB)
  Using cached FuzzyTM-2.0.5-py3-none-any.whl (29 kB)
  Using cached pyFUME-0.2.25-py3-none-any.whl (67 kB)
  Using cached fst_pso-1.8.1-py3-none-any.whl
  Using cached simpful-2.9.0-py3-none-any.whl (30 kB)
  Using cached miniful-0.0.6-py3-none-any.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires sklearn, which is not installed.


In [2]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string

In [3]:
# A function used to build a vocabulary based on descending word frequencies 
def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv

In [4]:
# A function used to learn word embeddings through Word2vec module
def get_embeddings(inp_data, vocabulary_inv, size_features=100,
                   mode='skipgram',
                   min_word_count=2,
                   context=5):
    model_name = "embedding"
    model_name = os.path.join(model_name)
    num_workers = 15  # Number of threads to run in parallel
    downsampling = 1e-3  # Downsample setting for frequent words
    print('Training Word2Vec model...')
    # use inp_data and vocabulary_inv to reconstruct sentences
    sentences = [[vocabulary_inv[w] for w in s] for s in inp_data]
    if mode == 'skipgram':
        sg = 1
        print('Model: skip-gram')
    elif mode == 'cbow':
        sg = 0
        print('Model: CBOW')
    embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                        sg=sg,
                                        vector_size=size_features,
                                        min_count=min_word_count,
                                        window=context,
                                        sample=downsampling)
    print("Saving Word2Vec model {}".format(model_name))
    embedding_weights = np.zeros((len(vocabulary_inv), size_features))
    for i in range(len(vocabulary_inv)):
        word = vocabulary_inv[i]
        if word in embedding_model.wv:
            embedding_weights[i] = embedding_model.wv[word]
        else:
            embedding_weights[i] = np.random.uniform(-0.25, 0.25,
                                                     embedding_model.vector_size)
    return embedding_weights

In [5]:
def preprocess_df(df):
    # get English stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    # prepare translation table to translate punctuation to space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    preprocessed_sentences = []
    for i, row in df.iterrows():
        sent = row["text"]
        sent_nopuncts = sent.translate(translator)
        words_list = sent_nopuncts.strip().split()
        filtered_words = [word for word in words_list if word not in stop_words and len(word) != 1] # also skip space from above translation
        preprocessed_sentences.append(" ".join(filtered_words))
    df["text"] = preprocessed_sentences
    return df

In [6]:
data_path = "/."

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train["text"] = df_train["review"]
df_test["text"] = df_test["review"]
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

# tokenization 
tagged_data = [word_tokenize(_d) for i, _d in enumerate(df_train["text"])]
# build vocabulary from tokenized data
word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_data)
# use the above mapping to create input data
inp_data = [[vocabulary[word] for word in text] for text in tagged_data]
# get embedding vector
embedding_weights = get_embeddings(inp_data, vocabulary_inv)


tagged_train_data = [word_tokenize(_d) for i, _d in enumerate(df_train["text"])]
tagged_test_data = [word_tokenize(_d) for i, _d in enumerate(df_test["text"])]

train_vec = []
for doc in tagged_train_data:
    vec = 0
    for w in doc:
        vec += embedding_weights[vocabulary[w]]
    vec = vec / len(doc)
    train_vec.append(vec)

test_vec = []
for doc in tagged_test_data:
    vec = 0
    length = 0
    for w in doc:
        try:
            vec += embedding_weights[vocabulary[w]]
            length += 1
        except:
            continue
    vec = vec / length
    test_vec.append(vec)

clf = LogisticRegression(max_iter=100000000).fit(train_vec, df_train["label"])
preds = clf.predict(test_vec)

# in your implemetation, create the output file using the same format
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(preds):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)

dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv("predicted.csv", index=False)

Training Word2Vec model...
Model: skip-gram
Saving Word2Vec model embedding


In [8]:
str.maketrans()

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,label,review,text
0,0,b'True',"b""{'romantic': False, 'intimate': False, 'clas...",b'15:0-2:0',b'44107',NaN,NaN,b'False',b'11:30-2:0',NaN,...,b'False',NaN,NaN,b'17800 Detroit Ave',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '16:0-2:0', 'Tuesday': '15:0-2:0', ...",american (traditional),"So, we stopped here on our way to the Side Que...",So stopped way Side Quest street nWe know expe...
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85042',NaN,NaN,b'True',b'11:0-20:30',NaN,...,b'False',NaN,b'False',"b'2160 E Baseline Rd, Ste 128'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",american (new),This is our go-to healthy spot! The food is al...,This go healthy spot The food always fresh del...
2,2,NaN,NaN,b'11:0-21:0',b'M4M 3G6',NaN,NaN,NaN,b'11:0-21:0',NaN,...,NaN,NaN,NaN,b'1000 Gerrard St E',NaN,NaN,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",mexican,Food court meal at Gerrard Square. It's been ...,Food court meal Gerrard Square It since nachos...
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",NaN,b'89146',NaN,NaN,b'False',NaN,NaN,...,b'True',NaN,NaN,b'6700 W Charleston Blvd',b'True',NaN,NaN,mexican,"Located on Rainbow/Charleston, this small fami...",Located Rainbow Charleston small family owned ...
4,4,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'44133',NaN,NaN,b'False',NaN,NaN,...,b'False',NaN,NaN,b'5630 Wallings Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...",NaN,chinese,No frills Chinese takeout joint which serves u...,No frills Chinese takeout joint serves best ar...
